# ✨ Préparation des features pour modèles ML
Ce notebook reproduit étape par étape le script `prepare_features.py`, avec une explication détaillée de chaque transformation.

## Objectifs
- Nettoyage, vectorisation, et encodage des variables
- Création des matrices d'entrée `X` et des cibles `y`
- Sauvegarde des features pour l'entraînement de modèles

## 📦 Import des bibliothèques nécessaires
On importe ici les librairies indispensables pour manipuler les données (Pandas, NumPy), vectoriser du texte (TfidfVectorizer), encoder les variables catégorielles (OneHotEncoder), et sauvegarder les objets pour les réutiliser dans l'entraînement.

**Choix des bibliothèques et raisons ?**

- **`pandas`** : pour la manipulation tabulaire efficace des données structurées (chargement CSV, nettoyage, sélection).
- **`numpy`** : pour la manipulation de vecteurs et matrices numériques, conversion de types, calculs rapides.
- **`os`** : permet de construire des chemins de fichiers dynamiques et compatibles tous systèmes (`os.path.join()`).
- **`sklearn.feature_extraction.text.TfidfVectorizer`** : convertit le texte brut en vecteurs numériques pondérés, très adapté pour des modèles ML supervisés sur des données textuelles.
- **`sklearn.preprocessing.OneHotEncoder`** : encode les variables catégorielles (comme le niveau vendeur) en vecteurs binaires, indispensable pour les algorithmes ML traditionnels.
- **`scipy.sparse`** : stocke efficacement en mémoire les matrices creuses générées par le TF-IDF ou le OneHot (économie mémoire et rapidité).
- **`joblib`** : bibliothèque optimisée pour sérialiser (sauvegarder/recharger) les objets Python lourds, comme les modèles entraînés ou les preprocessors.

In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
import os

## 📂 Chargement du dataset Fiverr nettoyé
Nous utilisons le fichier CSV issu de l'ingestion précédente. 
**Encodage 'latin-1'** choisi pour éviter les erreurs liées aux caractères spéciaux.  
Cette étape consiste à **importer les données nettoyées** depuis un fichier `.csv`, afin de pouvoir les manipuler avec `pandas`. Elle intervient **en tout début de pipeline**, juste après l'ingestion et le nettoyage initial, car toutes les étapes suivantes (vectorisation, encodage, prédiction) nécessitent un DataFrame propre et chargé en mémoire.

### Pourquoi cette étape maintenant ?
- Elle constitue le point d’entrée du pipeline de **prétraitement et d’inférence**.
- C’est à partir de ce dataset que nous allons appliquer les traitements (vectorisation, encodage), entraîner ou utiliser les modèles, et produire les prédictions.
- Il est important que cette étape soit réalisée **avant tout calcul ou transformation**, pour garantir que les données soient bien disponibles dans leur état attendu.

### Résultat attendu
- Le DataFrame `df` est chargé avec l’ensemble des lignes et colonnes du fichier CSV.
- C’est **la base de travail unique** utilisée pour le prétraitement, la vectorisation et les prédictions futures.
- On peut ensuite visualiser les premières lignes pour **vérifier visuellement la structure et le contenu**, détecter d’éventuelles erreurs, ou simplement comprendre les données disponibles.

In [5]:
# Chargement du fichier CSV dans un DataFrame pandas

df = pd.read_csv(
    'data/fiverr_gigs/fiverr-data-gigs-cleaned.csv', # Chemin relatif vers le fichier de données
    encoding='latin-1', # Utilisé pour décoder les caractères spéciaux (utile si UTF-8 génère des erreurs, fréquent avec des accents)
    low_memory=False # Force pandas à lire tout le fichier avant de deviner les types de colonnes (évite les avertissements ou erreurs)
)

# Affiche les 10 premières lignes du DataFrame pour en examiner un aperçu
df.head(10)

,Title,Seller Level,Average Rating,Number of Reviewers,Price (USD)
0,"I will do excel formulas, vba macros, charts, ...",Level 2 Seller,5.0,56,41.433333
1,"I will do data mining, web scraping, data extr...",Top Rated Seller,5.0,1k+,15.538095
2,"I will do web scraping , data scraping, scrapi...",Level 2 Seller,5.0,63,10.361905
3,"I will do python web scraping , data entry , a...",Level 2 Seller,5.0,266,5.180952
4,I will data scraping web site scraping and dat...,Level 2 Seller,4.9,275,20.719048
5,"I will do perfect web scraping, data mining fo...",Level 2 Seller,5.0,181,10.361905
6,I will write web scraper using python for you,Top Rated Seller,5.0,211,51.790476
7,"I will do web scraping, crawling and data mini...",Top Rated Seller,5.0,398,82.861905
8,"I will do website scraping, python scripts aut...",Level 2 Seller,5.0,61,20.719048
9,I will automate your documents with python scr...,Clients,5.0,16,103.580952


## 🧠 Vectorisation du texte (TF-IDF)
On transforme la colonne `Title` (texte libre) en vecteurs numériques.

**Pourquoi TF-IDF ?**
- Représente l'importance des mots dans le corpus
- Plus performant qu’un simple comptage (CountVectorizer)
- Adapté à des modèles linéaires et arbres de décision

In [ ]:
tfidf = TfidfVectorizer(max_features=2000, stop_words='english')
tfidf.fit(df['Title'].fillna(''))
X_title = tfidf.transform(df['Title'].fillna(''))

## 🔢 Encodage du niveau du vendeur
On encode la colonne `Seller Level` (catégorielle) avec un OneHotEncoder.

**Pourquoi OneHotEncoder ?**
- Convertit chaque niveau en une colonne binaire distincte
- Compatible avec la plupart des modèles supervisés

On gère également les valeurs inconnues (`handle_unknown='ignore'`).

In [ ]:
ohe = OneHotEncoder(sparse=True, handle_unknown='ignore')
X_level = ohe.fit_transform(df['Seller Level'].fillna('Unknown').values.reshape(-1, 1))

## 🔢 Nombre d’avis (Feature numérique)
On utilise la colonne `Number of Reviews` telle quelle, convertie en tableau sparse pour l’empilement final.

In [ ]:
X_reviews = sparse.csr_matrix(df['Number of Reviews'].fillna(0).values.reshape(-1, 1))

## 🧩 Construction de la matrice finale X
On combine les trois types de features :
- TF-IDF du titre
- One-hot du niveau
- Nombre d’avis

In [ ]:
X = sparse.hstack([X_title, X_level, X_reviews], format='csr')
print(f"Matrice finale X : {X.shape}")

## 🎯 Extraction des variables cibles (multi-sortie)
Nous voulons prédire :
- `Price` (régression)
- `Rating` (régression)
- `Seller Level` (classification)

Les deux premières sont regroupées dans une matrice Y pour un modèle `MultiOutputRegressor`.

In [ ]:
y_reg = df[['Price', 'Rating']].fillna(0)
y_class = df['Seller Level'].fillna('Unknown')

## 💾 Sauvegarde des objets (modèles et features)
**Pourquoi sauvegarder ici ?**
- Pour séparer les étapes (prétraitement, entraînement, déploiement)
- Pour réutiliser les objets dans `train_multioutput.py` ou `app_gradio.py`

In [ ]:
joblib.dump(tfidf, 'models/tfidf_vectorizer.pkl')
joblib.dump(ohe, 'models/ohe_encoder.pkl')
sparse.save_npz('data/X_features.npz', X)
y_reg.to_csv('data/y_reg.csv', index=False)
y_class.to_csv('data/y_class.csv', index=False)